# Prepare driver

In [2]:
! wget -P /home/jovyan https://repo1.maven.org/maven2/org/postgresql/postgresql/42.2.5/postgresql-42.2.5.jar

--2019-12-28 10:47:11--  https://repo1.maven.org/maven2/org/postgresql/postgresql/42.2.5/postgresql-42.2.5.jar
Resolving repo1.maven.org (repo1.maven.org)... 151.101.244.209
Connecting to repo1.maven.org (repo1.maven.org)|151.101.244.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 825943 (807K) [application/java-archive]
Saving to: ‘/home/jovyan/postgresql-42.2.5.jar.1’

postgresql-42.2.5.j 100%[===================>] 806.58K  2.03MB/s    in 0.4s    

2019-12-28 10:47:12 (2.03 MB/s) - ‘/home/jovyan/postgresql-42.2.5.jar.1’ saved [825943/825943]



# Start spark application

In [3]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--driver-class-path /home/jovyan/postgresql-42.2.5.jar --jars /home/jovyan/postgresql-42.2.5.jar pyspark-shell'

import pyspark

spark = pyspark.sql.SparkSession.builder \
        .master("local[1]") \
        .appName("snapshot") \
        .getOrCreate()

print("Application started")

Application started


# Warm-up spark

In [4]:
spark.sparkContext.range(1000).sum()
print("Spark application is ready for work")

Spark application is ready for work


# Read some PostgreSQL Data

In [8]:
customers = spark.read.format('jdbc').options(
        url = "jdbc:postgresql://postgres:5432/postgres?user=postgres&password=postgres&currentSchema=inventory",
        database='postgres',
        dbtable='customers'
    ).load()

products = spark.read.format('jdbc').options(
        url = "jdbc:postgresql://postgres:5432/postgres?user=postgres&password=postgres&currentSchema=inventory",
        database='postgres',
        dbtable='products'
    ).load()

orders = spark.read.format('jdbc').options(
        url = "jdbc:postgresql://postgres:5432/postgres?user=postgres&password=postgres&currentSchema=inventory",
        database='postgres',
        dbtable='orders'
    ).load()

customers.registerTempTable("customers")
products.registerTempTable("products")
orders.registerTempTable("orders")

print("Customers table")
customers.show(5)
print("Orders table")
orders.show(5)
print("Products table")
products.show(5)


Customers table
+----+----------+---------+--------------------+
|  id|first_name|last_name|               email|
+----+----------+---------+--------------------+
|1001|     Sally|   Thomas|sally.thomas@acme...|
|1002|    George|   Bailey|  gbailey@foobar.com|
|1003|    Edward|   Walker|       ed@walker.com|
|1004|      Anne|Kretchmar|  annek@noanswer.org|
+----+----------+---------+--------------------+

Orders table
+-----+----------+---------+--------+----------+
|   id|order_date|purchaser|quantity|product_id|
+-----+----------+---------+--------+----------+
|10001|2016-01-16|     1001|       1|       102|
|10002|2016-01-17|     1002|       2|       105|
|10003|2016-02-19|     1002|       2|       106|
|10004|2016-02-21|     1003|       1|       107|
+-----+----------+---------+--------+----------+

Products table
+---+------------------+--------------------+------+
| id|              name|         description|weight|
+---+------------------+--------------------+------+
|101|      

# Read and join the data

In [17]:
# please write your query here
query = """
select 
c.id,
c.first_name,
c.last_name,
sum(weight) as total_weight,
current_timestamp as load_dttm  
from customers c 
left join orders o on c.id = o.purchaser
left join products p on o.product_id = p.id
where c.id <= 1005
group by c.id,c.first_name,c.last_name,current_timestamp
"""
result = spark.sql(query)
result.write.format("parquet").save("/home/jovyan/weight_report")

AnalysisException: 'path file:/home/jovyan/weight_report already exists.;'

In [18]:
spark.read \
    .format("parquet").load("/home/jovyan/weight_report").show()

+----+----------+---------+------------+--------------------+
|  id|first_name|last_name|total_weight|           load_dttm|
+----+----------+---------+------------+--------------------+
|1002|    George|   Bailey|       1.875|2019-12-28 10:56:...|
|1003|    Edward|   Walker|         5.3|2019-12-28 10:56:...|
|1001|     Sally|   Thomas|         8.1|2019-12-28 10:56:...|
|1004|      Anne|Kretchmar|        null|2019-12-28 10:56:...|
+----+----------+---------+------------+--------------------+



# Testing

In [22]:

# please write your query here
query = """SELECT c.id, c.first_name, c.last_name, sum(p.weight) AS total_weight, current_timestamp() AS load_dttm
FROM customers c
LEFT JOIN orders o
ON c.id = o.purchaser
LEFT JOIN products p
ON o.product_id = p.id
GROUP BY c.id, c.first_name, c.last_name
HAVING c.id <= 1005
ORDER BY c.id"""

result = spark.sql(query)
result.write.format("parquet").save("/home/jovyan/weight_report_test")

AnalysisException: 'path file:/home/jovyan/weight_report_test already exists.;'

In [21]:
spark.read \
    .format("parquet").load("/home/jovyan/weight_report_test").show()

+----+----------+---------+------------+--------------------+
|  id|first_name|last_name|total_weight|           load_dttm|
+----+----------+---------+------------+--------------------+
|1003|    Edward|   Walker|         5.3|2019-12-28 11:11:...|
|1002|    George|   Bailey|       1.875|2019-12-28 11:11:...|
|1001|     Sally|   Thomas|         8.1|2019-12-28 11:11:...|
|1004|      Anne|Kretchmar|        null|2019-12-28 11:11:...|
+----+----------+---------+------------+--------------------+

